In [168]:
from yahoo_fin import stock_info as si
import yfinance as yf
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())
import numpy as np
import pandas_ta as ta
import pandas as pd
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
import pandas as pd
from GoogleNews import GoogleNews
googlenews = GoogleNews()
googlenews = GoogleNews(lang='en', region='US')
googlenews = GoogleNews(encode='utf-8')

In [169]:
#  Symbol
symbol = 'AAPL'
market_symbol = '^NDX'

In [170]:
def get_biz_days_delta_date(start_date_str, delta_days):
    start_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = start_date + (delta_days * US_BUSINESS_DAY)
    end_date_str = datetime.datetime.strftime(end_date, "%Y-%m-%d")
    return end_date_str

In [171]:
def load_price_data(symbol, start_date, end_date):
    # Download data
    try:
        df = si.get_data(symbol, start_date=start_date, end_date=end_date, index_as_date=False)
        return df
    except:
        print('Error loading stock data for ' + symbol)
        return None
    
today = datetime.datetime.today()
today_str = today.strftime("%Y-%m-%d")
past_date_str = get_biz_days_delta_date(today_str, -251)
df = load_price_data(symbol, past_date_str,today_str)  
market_df = load_price_data(market_symbol, past_date_str,today_str) 

In [172]:
def plot_market_graph(market_symbol, in_df):
    df = in_df.copy()
   
    fig = make_subplots(rows=1, cols=1,
                        subplot_titles=['Close'])

    #  Plot close price
    fig.add_trace(go.Line(x=df.index, y=df['close'], line=dict(color="blue", width=1), name="Close"), row=1, col=1)
    
    fig.update_layout(
        title={'text': market_symbol, 'x': 0.5},
        autosize=True,)
    fig.update_yaxes(range=[0, 1000000000], secondary_y=True)
    fig.update_yaxes(visible=False, secondary_y=True)  # hide range slider

    #  Save output
    fig.show()



In [173]:
def plot_stock_graph(symbol, in_df):
    df = in_df.copy()
    #  Calculate strategy indicators
    df.ta.macd(close='close', fast=12, slow=26, signal=9, append=True)
    df['RSI'] = ta.rsi(df['close'], length=14)
    indicator_bb = BollingerBands(close=df['close'], window=20, window_dev=2)
    df['BB_mid'] = indicator_bb.bollinger_mavg()
    df['BB_high'] = indicator_bb.bollinger_hband()
    df['BB_low'] = indicator_bb.bollinger_lband()
    indicator_keltner = KeltnerChannel(high=df['high'], low=df['low'], close=df['close'], window=20)
    df['Keltner_mid'] = indicator_keltner.keltner_channel_mband()
    df['Keltner_high'] = indicator_keltner.keltner_channel_hband()
    df['Keltner_low'] = indicator_keltner.keltner_channel_lband()
   
    fig = make_subplots(rows=5, cols=1,
                        subplot_titles=['Close', 'MACD', 'RSI', 'Bollinger Bands', 'Keltner Channels',])

    #  Plot close price
    fig.add_trace(go.Line(x=df.index, y=df['close'], line=dict(color="blue", width=1), name="Close"), row=1, col=1)

    # Plot MACD
    fig.add_trace(go.Line(x=df.index, y=df['MACD_12_26_9'], line=dict(color="#99b3ff", width=1), name="MACD"), row=2,
                  col=1)
    fig.add_trace(go.Line(x=df.index, y=df['MACDs_12_26_9'], line=dict(color="#ebab34", width=1), name="MACD"), row=2,
                  col=1)

    # Plot RSI
    fig.add_trace(go.Line(x=df.index, y=df['RSI'], line=dict(color="#99b3ff", width=1), name="RSI"), row=3, col=1)
    fig.add_hline(y=30, line=dict(color="#ebab34", width=1), row=3, col=1)
    fig.add_hline(y=70, line=dict(color="#ebab34", width=1), row=3, col=1)

    # Plot Bollinger
    fig.add_trace(go.Line(x=df.index, y=df['close'], line=dict(color="blue", width=1), name="Close"), row=4, col=1)
    fig.add_trace(go.Line(x=df.index, y=df['BB_high'], line=dict(color="#ebab34", width=1), name="BB High"), row=4, col=1)
    fig.add_trace(go.Line(x=df.index, y=df['BB_mid'], line=dict(color="#fac655", width=1), name="BB Mid"), row=4, col=1)
    fig.add_trace(go.Line(x=df.index, y=df['BB_low'], line=dict(color="#ebab34", width=1), name="BB Low"), row=4, col=1)

    # Plot Keltner
    fig.add_trace(go.Line(x=df.index, y=df['close'], line=dict(color="blue", width=1), name="Close"), row=5, col=1)
    fig.add_trace(go.Line(x=df.index, y=df['Keltner_high'], line=dict(color="#ebab34", width=1), name="Keltner High"),
                  row=5, col=1)
    fig.add_trace(go.Line(x=df.index, y=df['Keltner_mid'], line=dict(color="#fac655", width=1), name="Keltner Mid"),
                  row=5, col=1)
    fig.add_trace(go.Line(x=df.index, y=df['Keltner_low'], line=dict(color="#ebab34", width=1), name="Keltner Low"),
                  row=5, col=1)

    fig.update_layout(
        title={'text': symbol, 'x': 0.5},
        autosize=False, width= 800, height= 1600)
    fig.update_yaxes(range=[0, 1000000000], secondary_y=True)
    fig.update_yaxes(visible=False, secondary_y=True)  # hide range slider

    #  Save output
    fig.show()
    


In [174]:
def retrieve_google_news(symbol, past_days):
    googlenews = GoogleNews(period=f"{past_days}d")
    googlenews.clear()
    googlenews.search(symbol)
    result = googlenews.result()
    df = pd.DataFrame(result)
    return df

In [175]:
yf_symbol = yf.Ticker(symbol)

# Print company info
company_info = yf_symbol.info
#company_info_obj = json.loads(employee_string)
print(company_info["longName"])
print(company_info["longBusinessSummary"])
print("Website: ",company_info["website"])
print("Industry: ",company_info["industry"])
print("Current price: ",company_info["currentPrice"])
print("Total revenuee: ",locale.format_string("%d", company_info["totalRevenue"], grouping=True))
print("Gross profits: ",locale.format_string("%d", company_info["grossProfits"], grouping=True))
print("Revenue growth: ",company_info["revenueGrowth"])  
print("Earnings growth: ",company_info["earningsGrowth"])    
print("Profit margins: ",company_info["profitMargins"])
print("Debt to equity: ",company_info["debtToEquity"])     
print("Return on equity: ",company_info["returnOnEquity"])     
print("Quick ratio: ",company_info["quickRatio"])  
print("52-Week high: ",company_info["fiftyTwoWeekHigh"])      
print("52-Week low: ",company_info["fiftyTwoWeekLow"])       

Apple Inc.
Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. In addition, the company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; AirPods Max, an over-ear wireless headphone; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, and iPod touch. Further, it provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. Additionally, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand radio stations; Apple News+, a subscription news and magazine service; App

In [176]:
#  Plot stock graph and TAs
plot_stock_graph(symbol, df)

C:\Users\dutch\anaconda3\envs\tradesystem1\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [177]:
#  Plot market graph
plot_market_graph(market_symbol, market_df)

In [178]:
#  Retrieve Google News
news_df = retrieve_google_news(symbol, 5)

for index, row in news_df.iterrows():
    print(f"-- {row['title']} --\n{row['desc']}\n\n")

-- Apple whistleblower brings spying claims to UK --
Britain's information watchdog is investigating claims that Apple was able 
to access personal information on workers' phones after a privacy complaint 
was...


-- A device unique to the company was discovered on the Apple website --
Apple may soon release a new version of its proprietary USB Type-C power 
adapter with a notable improvement. A device unique to the company was 
discovered...


-- Apple likely working on Dual USB-C Power Adapter, says report --
Tech giant Apple is likely working on a dual-port USB-C wall charger and it 
may see a release in the near future as an analyst has claimed it will soon 
be...


-- Https Ecommerce.apple.com Content B2b Static En Us Dfl-seg Contact.html? – ictsd.org --
The Apple Business Essentials price and key information are summarized as 
follows: ($2). The cost for a single device, as well as 50GB of storage,...


-- Apple Has Released A Report Detailing Just How Well Third-Party Apps Perf